- Vary sigma_IOU and see how it changes the counts

In [27]:
%reset -f

import os
import pandas as pd
import bz2
import ujson
from scripts.GTCompare import import_analyze_export as iae

current_directory = "Z:/04_Daten/GroundThruth/Fussverkehrausfkommen/OTC07_23-09-19_17-00-00/vary_sigmaIOU"
# current_directory = "Z:/04_Daten/GroundThruth/Fussverkehrausfkommen/OTC13_23-10-22_17-00-00/vary_SigmaIOU"

def extract_data_from_otevents(otevents_file):
    # Dateiinhalt einlesen
    with bz2.open(otevents_file, "rt", encoding="UTF-8") as file:
        dictfile = ujson.load(file)
    
    # Convert to DataFrame
    df = pd.DataFrame.from_dict(dictfile['event_list'])
    df["Sigma_IOU"] = os.path.splitext(os.path.basename(otevents_file))[0]
    return df

def process_otevents_files(directory="."):
    # Durchsuche das angegebene Verzeichnis nach Dateien mit der Endung ".otevents"
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".otevents"):
                # Extrahiere den Dateinamen (ohne Erweiterung)
                filename = os.path.splitext(file)[0]
                
                # Extrahiere die Daten aus der .otevents-Datei und erstelle ein DataFrame
                df = extract_data_from_otevents(os.path.join(root, file))
                globals()[filename] = df

# OTC data
process_otevents_files(current_directory)

# GT Data
def events_to_df(filepath, key = 'event_list'):
    # Open otevents-file
    with bz2.open(filepath, "rt", encoding="UTF-8") as file:
        dictfile = ujson.load(file)
    
    # Convert to DataFrame
    EVENTS = pd.DataFrame.from_dict(dictfile[key])

    return EVENTS

GT_events = pd.DataFrame()
for root, dirs, files in os.walk(current_directory):
    for file in files:
        if file.endswith('.otgtevents'):
            path = os.path.join(root, file)
            GT_eventfile = events_to_df(path)
            GT_eventfile['File'] = file
            GT_events = pd.concat([GT_events, GT_eventfile])
            del GT_eventfile, root, dirs, files, path

In [29]:
import numpy as np

def import_analyze_export(OTCEvents, GTEvents, Sigma = "", include_GT = True):
   
    # Häufigkeit der road_user_id: Jede nur einmal
    enter_events = OTCEvents[OTCEvents['event_type'] == 'enter-scene'].drop_duplicates('road_user_id')
    leave_events = OTCEvents[OTCEvents['event_type'] == 'leave-scene'].drop_duplicates('road_user_id')
    section_events = OTCEvents[OTCEvents['event_type'] == 'section-enter'].drop_duplicates(['road_user_id', 'section_id'])
    section_events = section_events.reset_index(drop=True)

    sections = pd.DataFrame(section_events.groupby(['road_user_id'])['section_id'].unique())
    sections = sections.rename(columns={'section_id': 'sections'})
    try:
        sections[['section_1', 'section_2', 'section_3']] = pd.DataFrame(sections['sections'].tolist(), index=sections.index)
        # del sections['sections']
        
        section_events = pd.merge(section_events, sections, how='left', on='road_user_id')
        section_events[['section_1', 'section_2', 'section_3']] = section_events[['section_1', 'section_2', 'section_3']].replace({None: np.NaN})

        OTA_events_filtered = pd.concat([enter_events, section_events], ignore_index=True)
        del enter_events, leave_events, section_events, sections 

        # Eventklassifizierung
        OTA_events_filtered.loc[(OTA_events_filtered['section_1'].isna()) & (OTA_events_filtered['section_2'].isna()), 'event_type2'] = 'No Section'
        OTA_events_filtered.loc[~(OTA_events_filtered['section_1'].isna()) & (OTA_events_filtered['section_2'].isna()), 'event_type2'] = 'Section'
        OTA_events_filtered.loc[(OTA_events_filtered['section_1'].isna()) & ~(OTA_events_filtered['section_2'].isna()), 'event_type2'] = 'Section'
        OTA_events_filtered.loc[~(OTA_events_filtered['section_1'].isna()) & ~(OTA_events_filtered['section_2'].isna()), 'event_type2'] = 'Flow'

        OTA_events_filtered.loc[(OTA_events_filtered['section_1'].isna()) & (OTA_events_filtered['section_2'].isna()) & (OTA_events_filtered['section_3'].isna()), 'event_type2'] = 'No Section'
        OTA_events_filtered.loc[~(OTA_events_filtered['section_1'].isna()) & (OTA_events_filtered['section_2'].isna()) & (OTA_events_filtered['section_3'].isna()), 'event_type2'] = 'Section'
        OTA_events_filtered.loc[(OTA_events_filtered['section_1'].isna()) & ~(OTA_events_filtered['section_2'].isna()) & (OTA_events_filtered['section_3'].isna()), 'event_type2'] = 'Section'
        OTA_events_filtered.loc[(OTA_events_filtered['section_1'].isna()) & (OTA_events_filtered['section_2'].isna()) & ~(OTA_events_filtered['section_3'].isna()), 'event_type2'] = 'Section'
        
        OTA_events_filtered.loc[~(OTA_events_filtered['section_1'].isna()) & ~(OTA_events_filtered['section_2'].isna()) & (OTA_events_filtered['section_3'].isna()), 'event_type2'] = 'Flow'
        OTA_events_filtered.loc[~(OTA_events_filtered['section_1'].isna()) & (OTA_events_filtered['section_2'].isna()) & ~(OTA_events_filtered['section_3'].isna()), 'event_type2'] = 'Flow'
        OTA_events_filtered.loc[(OTA_events_filtered['section_1'].isna()) & ~(OTA_events_filtered['section_2'].isna()) & ~(OTA_events_filtered['section_3'].isna()), 'event_type2'] = 'Flow'    
    
    except:
        sections[['section_1', 'section_2']] = pd.DataFrame(sections['sections'].tolist(), index=sections.index)
        # del sections['sections']

        section_events = pd.merge(section_events, sections, how='left', on='road_user_id')
        section_events[['section_1', 'section_2']] = section_events[['section_1', 'section_2']].replace({None: np.NaN})

        OTA_events_filtered = pd.concat([enter_events, section_events], ignore_index=True)
        del enter_events, leave_events, section_events, sections 

        # Eventklassifizierung
        OTA_events_filtered.loc[(OTA_events_filtered['section_1'].isna()) & (OTA_events_filtered['section_2'].isna()), 'event_type2'] = 'No Section'
        OTA_events_filtered.loc[~(OTA_events_filtered['section_1'].isna()) & (OTA_events_filtered['section_2'].isna()), 'event_type2'] = 'Section'
        OTA_events_filtered.loc[(OTA_events_filtered['section_1'].isna()) & ~(OTA_events_filtered['section_2'].isna()), 'event_type2'] = 'Section'
        OTA_events_filtered.loc[~(OTA_events_filtered['section_1'].isna()) & ~(OTA_events_filtered['section_2'].isna()), 'event_type2'] = 'Flow'
        
    OTA_events_filtered = OTA_events_filtered[['road_user_id', 'road_user_type', 'frame_number', 'section_id', 'event_type2']]
    
    # Für road_user_ids, die eine Section schneiden sollen die enter-scene-Events entfernt werden
    temp = OTA_events_filtered[OTA_events_filtered['event_type2'] != 'No Section'].reset_index(drop=True)

    temp['dummy'] = 1
    temp = temp[['road_user_id', 'dummy']].drop_duplicates()
    OTA_events_filtered = pd.merge(OTA_events_filtered, temp, how='left', on='road_user_id')

    OTA_events_filtered = OTA_events_filtered.drop(OTA_events_filtered[(OTA_events_filtered['section_id'].isna()) & (~OTA_events_filtered['dummy'].isna())].index)
    OTA_events_filtered = OTA_events_filtered.sort_values(by=['road_user_id', 'frame_number'])

    OTA_events_filtered = OTA_events_filtered.drop(OTA_events_filtered[OTA_events_filtered.duplicated(subset=['road_user_id'], keep='last')].index)

    OTA_events_filtered.loc[(OTA_events_filtered['section_id'] == '1') & (OTA_events_filtered['event_type2'] == 'Section'), 'event_type3'] = 'Section 1'
    OTA_events_filtered.loc[(OTA_events_filtered['section_id'] == '2') & (OTA_events_filtered['event_type2'] == 'Section'), 'event_type3'] = 'Section 2'
    OTA_events_filtered.loc[(OTA_events_filtered['section_id'] == '3') & (OTA_events_filtered['event_type2'] == 'Section'), 'event_type3'] = 'Section 3'
    OTA_events_filtered.loc[(OTA_events_filtered['section_id'] == '1') & (OTA_events_filtered['event_type2'] == 'Flow'), 'event_type3'] = 'Flow S1'
    OTA_events_filtered.loc[(OTA_events_filtered['section_id'] == '2') & (OTA_events_filtered['event_type2'] == 'Flow'), 'event_type3'] = 'Flow S2'
    OTA_events_filtered.loc[(OTA_events_filtered['section_id'] == '3') & (OTA_events_filtered['event_type2'] == 'Flow'), 'event_type3'] = 'Flow S3'
    OTA_events_filtered.loc[(OTA_events_filtered['event_type2'] == 'No Section'), 'event_type3'] = 'No Section'
    
    
    GT_events = GTEvents
    GT_events['event_type3'] = 'Flow'
    GT_events = GT_events.drop_duplicates(subset='road_user_id', keep='first')


    # # Auswertung
    OTACounts = pd.DataFrame(OTA_events_filtered.groupby(['road_user_type', 'event_type3'])['road_user_id'].count()).reset_index().rename(columns = {'road_user_id': 'OTAnalytics_0'})
    OTACounts['Sigma_IOU'] = "0," + Sigma
    GTCounts = pd.DataFrame(GT_events.groupby(['road_user_type', 'event_type3'])['road_user_id'].count()).reset_index().rename(columns = {'road_user_id': 'OTGroundTruth'})

    Counts = pd.merge(OTACounts, GTCounts, how='outer', left_on=['road_user_type', 'event_type3'], right_on = ['road_user_type', 'event_type3'])
    Counts['OTGroundTruth'] = Counts['OTGroundTruth'].astype('Int64')
    Counts['Differenz'] = Counts['OTGroundTruth'] - Counts['OTAnalytics_0']
    Counts['Diff [%]'] = np.round((1 - np.round(Counts['OTAnalytics_0'] / Counts['OTGroundTruth'], 3)) * 100, 1)

    # print(Counts)
    
    if include_GT == False:
        return OTACounts
    else:
        return Counts
    
    # if savename != "":
    #     Counts.to_csv(directory + 'Counts_' + Fall +'.csv', sep=';')

Counts_events_sigmaIOU_30 = import_analyze_export(events_sigmaIOU_30, GT_events, Sigma = "30", include_GT = True)
Counts_events_sigmaIOU_32 = import_analyze_export(events_sigmaIOU_32, GT_events, Sigma = "32", include_GT = True)
Counts_events_sigmaIOU_34 = import_analyze_export(events_sigmaIOU_34, GT_events, Sigma = "34", include_GT = True)
Counts_events_sigmaIOU_36 = import_analyze_export(events_sigmaIOU_36, GT_events, Sigma = "36", include_GT = True)
Counts_events_sigmaIOU_38 = import_analyze_export(events_sigmaIOU_38, GT_events, Sigma = "38", include_GT = True)
Counts_events_sigmaIOU_40 = import_analyze_export(events_sigmaIOU_40, GT_events, Sigma = "40", include_GT = True)
Counts_events_sigmaIOU_42 = import_analyze_export(events_sigmaIOU_42, GT_events, Sigma = "42", include_GT = True)
Counts_events_sigmaIOU_44 = import_analyze_export(events_sigmaIOU_44, GT_events, Sigma = "44", include_GT = True)
Counts_events_sigmaIOU_46 = import_analyze_export(events_sigmaIOU_46, GT_events, Sigma = "46", include_GT = True)

Counts_events = pd.concat([Counts_events_sigmaIOU_30,
                           Counts_events_sigmaIOU_32,
                           Counts_events_sigmaIOU_34,
                           Counts_events_sigmaIOU_36,
                           Counts_events_sigmaIOU_38,
                           Counts_events_sigmaIOU_40,
                           Counts_events_sigmaIOU_42,
                           Counts_events_sigmaIOU_44,
                           Counts_events_sigmaIOU_46])

# Counts_events_sigmaIOU_20 = import_analyze_export(events_sigmaIOU_20, GT_events, Sigma = "20", include_GT = True)
# Counts_events_sigmaIOU_24 = import_analyze_export(events_sigmaIOU_24, GT_events, Sigma = "24", include_GT = True)
# Counts_events_sigmaIOU_28 = import_analyze_export(events_sigmaIOU_28, GT_events, Sigma = "28", include_GT = True)
# Counts_events_sigmaIOU_32 = import_analyze_export(events_sigmaIOU_32, GT_events, Sigma = "32", include_GT = True)
# Counts_events_sigmaIOU_36 = import_analyze_export(events_sigmaIOU_36, GT_events, Sigma = "36", include_GT = True)
# Counts_events_sigmaIOU_40 = import_analyze_export(events_sigmaIOU_40, GT_events, Sigma = "40", include_GT = True)
# Counts_events_sigmaIOU_44 = import_analyze_export(events_sigmaIOU_44, GT_events, Sigma = "44", include_GT = True)
# Counts_events_sigmaIOU_48 = import_analyze_export(events_sigmaIOU_48, GT_events, Sigma = "48", include_GT = True)


# Counts_events = pd.concat([Counts_events_sigmaIOU_20,
#                            Counts_events_sigmaIOU_24,
#                            Counts_events_sigmaIOU_28,
#                            Counts_events_sigmaIOU_32,
#                            Counts_events_sigmaIOU_36,
#                            Counts_events_sigmaIOU_40,
#                            Counts_events_sigmaIOU_44,
#                            Counts_events_sigmaIOU_48])

# Counts_events.to_csv(current_directory + '/Counts_' + 'events' +'.csv', sep=';')
# del events_sigmaIOU_30, events_sigmaIOU_32, events_sigmaIOU_34, events_sigmaIOU_36, events_sigmaIOU_38, events_sigmaIOU_40, events_sigmaIOU_42, events_sigmaIOU_44, events_sigmaIOU_46, Counts_events_sigmaIOU_30, Counts_events_sigmaIOU_32, Counts_events_sigmaIOU_34, Counts_events_sigmaIOU_36, Counts_events_sigmaIOU_38, Counts_events_sigmaIOU_40, Counts_events_sigmaIOU_42, Counts_events_sigmaIOU_44, Counts_events_sigmaIOU_46

In [30]:
import plotly.express as px
from plotly.graph_objs import *
import plotly.io as pio

data = Counts_events.rename(columns={'event_type3': 'Event', 'OTAnalytics_0': 'Anzahl'})


fig = px.bar(data[data['road_user_type'] == 'car'], x='Sigma_IOU', y='Anzahl',
             color='Event', 
            #  barmode='group',
            #  category_orders='road_user_type'
            )
# fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
fig.show()



In [31]:
import plotly.express as px
import nbformat


fig = px.bar(data[data['road_user_type'] == 'pedestrian'], x='Sigma_IOU', y='Anzahl',
             color='Event', 
            #  barmode='group',
            #  category_orders='road_user_type'
            )
fig.show()